# 多普勒风廓线激光雷达 RWS 径向风速分析

## Notebook 说明

本 Notebook 提供交互式的 RWS（径向风速）分析环境，基于 `analysis_rws.py` 脚本实现。

### 功能包括：
1. 单角度组合分析
2. 多角度组合对比
3. CNR 质量控制
4. 完整的可视化输出

### 使用方法：
1. 依次执行各个代码单元
2. 可以修改参数进行自定义分析
3. 图表将直接显示在 Notebook 中

## 1. 导入库和配置

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['figure.figsize'] = (12, 6)

# 忽略警告
warnings.filterwarnings('ignore')

# 配置 matplotlib 在 Notebook 中显示图表
%matplotlib inline

print("✓ 库导入成功")

## 2. 加载数据

In [ ]:
# ==========================================
# 配置参数 / Configuration Parameters
# ==========================================
# 修改此处指定要分析的数据文件
# Modify here to specify the data file to analyze

data_file = 'Molas3D_00941_RealTime_20251005_前5000行.csv'
# 或使用另一个设备数据 / Or use another device data:
# data_file = 'Molas3D_00943_RealTime_20251005_前5000行.csv'

# ==========================================

# 加载数据
print(f"正在加载数据: {data_file}")
df = pd.read_csv(data_file)
df['Timestamp'] = pd.to_datetime(df['Timestamp'])

print(f"\n✓ 数据加载完成！共 {len(df)} 行数据")
print(f"时间范围: {df['Timestamp'].min()} 到 {df['Timestamp'].max()}")

# 显示前几行
df.head()

## 3. 数据概览

In [ ]:
print("="*60)
print("数据概览")
print("="*60)

# 基本统计
print(f"总数据点数: {len(df)}")
print(f"唯一时间戳数: {df['Timestamp'].nunique()}")

# 角度信息
print(f"\n方位角范围: {df['Azimuth(deg)'].min():.3f}° ~ {df['Azimuth(deg)'].max():.3f}°")
print(f"唯一方位角数: {df['Azimuth(deg)'].nunique()}")

print(f"\n仰角范围: {df['Elevation(deg)'].min():.3f}° ~ {df['Elevation(deg)'].max():.3f}°")
print(f"唯一仰角数: {df['Elevation(deg)'].nunique()}")

# 距离信息
print(f"\n距离范围: {df['Distance(m)'].min():.1f} m ~ {df['Distance(m)'].max():.1f} m")
print(f"距离门数量: {df['Distance(m)'].nunique()}")

# RWS 信息
print(f"\nRWS 范围: {df['RWS(m/s)'].min():.3f} m/s ~ {df['RWS(m/s)'].max():.3f} m/s")
print(f"RWS 均值: {df['RWS(m/s)'].mean():.3f} m/s")
print(f"RWS 标准差: {df['RWS(m/s)'].std():.3f} m/s")

# CNR 信息
print(f"\nCNR 范围: {df['CNR(dB)'].min():.3f} dB ~ {df['CNR(dB)'].max():.3f} dB")
print(f"CNR 均值: {df['CNR(dB)'].mean():.3f} dB")

## 4. 单角度组合分析

### 4.1 选择分析角度

In [ ]:
# 获取唯一的角度
azimuths = sorted(df['Azimuth(deg)'].unique())
elevations = sorted(df['Elevation(deg)'].unique())

print(f"可用方位角: {[f'{az:.2f}°' for az in azimuths[:5]]} ... (共 {len(azimuths)} 个)")
print(f"可用仰角: {[f'{el:.2f}°' for el in elevations]}")

# 选择第一个角度组合进行分析
test_azimuth = azimuths[0]
test_elevation = elevations[0]

print(f"\n选择角度组合: 方位角={test_azimuth:.3f}°, 仰角={test_elevation:.3f}°")

### 4.2 统计指标

In [ ]:
# 筛选特定角度组合的数据
mask = (np.abs(df['Azimuth(deg)'] - test_azimuth) < 0.1) & \
       (np.abs(df['Elevation(deg)'] - test_elevation) < 0.1)
data = df[mask]['RWS(m/s)']

# 计算统计量
stats = {
    '数据点数': len(data),
    '均值': data.mean(),
    '中位数': data.median(),
    '标准差': data.std(),
    '最小值': data.min(),
    '最大值': data.max(),
    'P10分位数': data.quantile(0.1),
    'P50分位数': data.quantile(0.5),
    'P90分位数': data.quantile(0.9)
}

print("="*60)
print(f"角度组合: 方位角={test_azimuth}°, 仰角={test_elevation}°")
print("="*60)
for key, value in stats.items():
    if isinstance(value, (int, np.integer)):
        print(f"{key}: {value}")
    else:
        print(f"{key}: {value:.3f}")

### 4.3 RWS 随距离的变化

In [ ]:
# 按距离分组统计
mask = (np.abs(df['Azimuth(deg)'] - test_azimuth) < 0.1) & \
       (np.abs(df['Elevation(deg)'] - test_elevation) < 0.1)
data = df[mask].copy()

distance_stats = data.groupby('Distance(m)')['RWS(m/s)'].agg([
    ('均值', 'mean'),
    ('标准差', 'std'),
    ('最小值', 'min'),
    ('最大值', 'max')
]).reset_index()

# 创建图表
fig, axes = plt.subplots(2, 1, figsize=(14, 10))

# 子图1: RWS 均值随距离变化
ax1 = axes[0]
ax1.plot(distance_stats['Distance(m)'], distance_stats['均值'], 
         'b-', linewidth=2, label='均值')
ax1.fill_between(distance_stats['Distance(m)'],
                  distance_stats['均值'] - distance_stats['标准差'],
                  distance_stats['均值'] + distance_stats['标准差'],
                  alpha=0.3, label='±1标准差')
ax1.set_xlabel('距离 (m)', fontsize=12)
ax1.set_ylabel('RWS (m/s)', fontsize=12)
ax1.set_title(f'RWS 随距离变化 - 方位角={test_azimuth}°, 仰角={test_elevation}°', 
              fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3)
ax1.legend(fontsize=10)

# 子图2: RWS 范围
ax2 = axes[1]
ax2.fill_between(distance_stats['Distance(m)'],
                  distance_stats['最小值'],
                  distance_stats['最大值'],
                  alpha=0.5, color='green', label='最小值-最大值范围')
ax2.plot(distance_stats['Distance(m)'], distance_stats['均值'],
         'r-', linewidth=2, label='均值')
ax2.set_xlabel('距离 (m)', fontsize=12)
ax2.set_ylabel('RWS (m/s)', fontsize=12)
ax2.set_title(f'RWS 变化范围 - 方位角={test_azimuth}°, 仰角={test_elevation}°', 
              fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3)
ax2.legend(fontsize=10)

plt.tight_layout()
plt.show()

### 4.4 RWS 分布特征

In [ ]:
mask = (np.abs(df['Azimuth(deg)'] - test_azimuth) < 0.1) & \
       (np.abs(df['Elevation(deg)'] - test_elevation) < 0.1)
data = df[mask]['RWS(m/s)'].dropna()

# 创建图表
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# 子图1: 直方图
ax1 = axes[0]
ax1.hist(data, bins=50, color='skyblue', edgecolor='black', alpha=0.7)
ax1.axvline(data.mean(), color='red', linestyle='--', linewidth=2, label=f'均值: {data.mean():.2f}')
ax1.axvline(data.median(), color='green', linestyle='--', linewidth=2, label=f'中位数: {data.median():.2f}')
ax1.set_xlabel('RWS (m/s)', fontsize=12)
ax1.set_ylabel('频数', fontsize=12)
ax1.set_title('RWS 直方图分布', fontsize=14, fontweight='bold')
ax1.legend(fontsize=10)
ax1.grid(True, alpha=0.3)

# 子图2: 箱线图
ax2 = axes[1]
bp = ax2.boxplot([data], vert=True, patch_artist=True, 
                  widths=0.5, showmeans=True,
                  boxprops=dict(facecolor='lightblue', alpha=0.7),
                  medianprops=dict(color='red', linewidth=2),
                  meanprops=dict(marker='D', markerfacecolor='green', markersize=8))
ax2.set_ylabel('RWS (m/s)', fontsize=12)
ax2.set_title('RWS 箱线图', fontsize=14, fontweight='bold')
ax2.set_xticklabels([f'Az={test_azimuth}°\nEl={test_elevation}°'])
ax2.grid(True, alpha=0.3, axis='y')

# 子图3: 分位数曲线
ax3 = axes[2]
quantiles = np.linspace(0, 1, 101)
quantile_values = [data.quantile(q) for q in quantiles]
ax3.plot(quantiles * 100, quantile_values, 'b-', linewidth=2)
ax3.axhline(data.quantile(0.1), color='orange', linestyle='--', linewidth=1.5, 
            label=f'P10: {data.quantile(0.1):.2f}')
ax3.axhline(data.quantile(0.5), color='green', linestyle='--', linewidth=1.5,
            label=f'P50: {data.quantile(0.5):.2f}')
ax3.axhline(data.quantile(0.9), color='red', linestyle='--', linewidth=1.5,
            label=f'P90: {data.quantile(0.9):.2f}')
ax3.set_xlabel('分位数 (%)', fontsize=12)
ax3.set_ylabel('RWS (m/s)', fontsize=12)
ax3.set_title('RWS 分位数曲线', fontsize=14, fontweight='bold')
ax3.legend(fontsize=10)
ax3.grid(True, alpha=0.3)

plt.suptitle(f'RWS 分布特征 - 方位角={test_azimuth}°, 仰角={test_elevation}°', 
             fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

## 5. 多角度组合对比分析

### 5.1 不同方位角对比

In [ ]:
# 固定仰角，对比不同方位角
elevation = elevations[0]
mask = np.abs(df['Elevation(deg)'] - elevation) < 0.1
data = df[mask].copy()

azimuths = sorted(data['Azimuth(deg)'].unique())

# 创建图表
fig, axes = plt.subplots(1, 2, figsize=(16, 6))

# 子图1: 箱线图
ax1 = axes[0]
data_list = [data[np.abs(data['Azimuth(deg)'] - az) < 0.1]['RWS(m/s)'].values 
             for az in azimuths]
labels = [f'{az:.1f}°' for az in azimuths]
bp = ax1.boxplot(data_list, labels=labels, patch_artist=True, showmeans=True,
                 boxprops=dict(facecolor='lightblue', alpha=0.7),
                 medianprops=dict(color='red', linewidth=2),
                 meanprops=dict(marker='D', markerfacecolor='green', markersize=6))
ax1.set_xlabel('方位角', fontsize=12)
ax1.set_ylabel('RWS (m/s)', fontsize=12)
ax1.set_title(f'不同方位角 RWS 分布对比 (仰角={elevation}°)', 
              fontsize=14, fontweight='bold')
ax1.grid(True, alpha=0.3, axis='y')
ax1.tick_params(axis='x', rotation=45)

# 子图2: 小提琴图
ax2 = axes[1]
plot_data = []
for az in azimuths:
    az_data = data[np.abs(data['Azimuth(deg)'] - az) < 0.1]['RWS(m/s)'].values
    for val in az_data:
        plot_data.append({'Azimuth': f'{az:.1f}°', 'RWS': val})
plot_df = pd.DataFrame(plot_data)

sns.violinplot(data=plot_df, x='Azimuth', y='RWS', ax=ax2, palette='Set2')
ax2.set_xlabel('方位角', fontsize=12)
ax2.set_ylabel('RWS (m/s)', fontsize=12)
ax2.set_title(f'不同方位角 RWS 分布（小提琴图）(仰角={elevation}°)', 
              fontsize=14, fontweight='bold')
ax2.grid(True, alpha=0.3, axis='y')
ax2.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

### 5.2 方位角 × 距离热力图

In [ ]:
# 固定仰角
elevation = elevations[0]
mask = np.abs(df['Elevation(deg)'] - elevation) < 0.1
data = df[mask].copy()

# 创建透视表
pivot_table = data.pivot_table(
    values='RWS(m/s)',
    index='Distance(m)',
    columns='Azimuth(deg)',
    aggfunc='mean'
)

# 创建热力图
fig, ax = plt.subplots(figsize=(14, 10))

im = ax.imshow(pivot_table.values, aspect='auto', cmap='coolwarm', 
               origin='lower', interpolation='nearest')

# 设置坐标轴
ax.set_xticks(np.arange(len(pivot_table.columns)))
ax.set_xticklabels([f'{az:.1f}' for az in pivot_table.columns], rotation=45)
ax.set_yticks(np.arange(0, len(pivot_table.index), 20))
ax.set_yticklabels([f'{pivot_table.index[i]:.0f}' for i in range(0, len(pivot_table.index), 20)])

ax.set_xlabel('方位角 (°)', fontsize=12)
ax.set_ylabel('距离 (m)', fontsize=12)
ax.set_title(f'RWS 热力图：方位角 × 距离 (仰角={elevation}°)', 
             fontsize=14, fontweight='bold')

# 添加颜色条
cbar = plt.colorbar(im, ax=ax)
cbar.set_label('RWS (m/s)', fontsize=12)

plt.tight_layout()
plt.show()

### 5.3 风速玫瑰图

In [ ]:
# 计算每个方位角的平均 RWS
azimuth_stats = df.groupby('Azimuth(deg)')['RWS(m/s)'].agg([
    ('均值', 'mean'),
    ('绝对值均值', lambda x: np.abs(x).mean())
]).reset_index()

# 转换为极坐标
theta = np.deg2rad(azimuth_stats['Azimuth(deg)'])
r = azimuth_stats['绝对值均值']

# 创建极坐标图
fig = plt.figure(figsize=(10, 10))
ax = fig.add_subplot(111, projection='polar')

# 设置0度在正北方向
ax.set_theta_zero_location('N')
ax.set_theta_direction(-1)  # 顺时针

# 绘制玫瑰图
bars = ax.bar(theta, r, width=np.deg2rad(5), 
              color='skyblue', alpha=0.7, edgecolor='blue', linewidth=1.5)

# 添加数值标签
for angle, radius in zip(theta, r):
    ax.text(angle, radius, f'{radius:.1f}', 
            ha='center', va='bottom', fontsize=8)

ax.set_title('RWS 方位玫瑰图（显示 |RWS| 均值）', 
             fontsize=14, fontweight='bold', pad=20)
ax.set_ylabel('|RWS| 均值 (m/s)', fontsize=12)

plt.tight_layout()
plt.show()

## 6. 总结

本 Notebook 完成了基于 RWS 的完整分析，包括：

1. ✓ 单角度组合统计分析
2. ✓ 距离维度变化趋势
3. ✓ 分布特征可视化
4. ✓ 多角度对比分析
5. ✓ 二维热力图
6. ✓ 风速玫瑰图

### 后续工作建议：

- 加载其他设备数据（如 00943）进行对比
- 分析不同时间段的 RWS 变化
- 结合气象参数进行关联分析
- 开发风场反演算法（VAD）